<a href="https://colab.research.google.com/github/samarakoon-ryan/resampling-unbalanced-datasets/blob/create-notebook/project_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Resampling using Credit Card Fraud Detection Dataset

In [1]:
# Import dependencies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import math

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline 
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn import config_context
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, roc_curve

In this project I will show an example of how resampling can be useful for unbalanced datasets in binary classification problems. I'll be using a Logistic Regression Model to demonstrate this. I strongly recommend that you use a combination of different methods to deal with unbalanced datasets, but would like to demonstrate the pros and cons of resampling.